# Imports and configuration

In [1]:
# Imports
import io
import math
import os

import folium
import geojson
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image

In [2]:
# Change directory to root
directory = os.getcwd().split('/')[-1]

if directory == 'notebooks':
    %cd ..

/Users/mschjolber001/repos/gecco-2023-ambulance-allocation/scripts


In [3]:
# Matplotlib import and setup
import matplotlib

# matplotlib.use('PDF')

import matplotlib.pyplot as plt
print(f'matplotlib backend: {matplotlib.get_backend()}')

matplotlib backend: module://matplotlib_inline.backend_inline


In [4]:
# configuration variables
should_save = False

In [5]:
# Import functions
from coordinate_converter import utm_to_longitude_latitude
import map_tools
import geojson_tools
import colors
import styles
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


In [6]:
incidents = pd.read_csv('proprietary_data/processed_data.csv', index_col=0)
incidents

,xcoor,ycoor,hastegrad,tiltak_type,rykker_ut,ank_hentested,avg_hentested,ledig_ikketransport,ledig_transport,non_transporting_vehicles,transporting_vehicles
tidspunkt,,,,,,,,,,,
2016-12-31 20:05:50,272500,6649500,H,Ambulanse,2016-12-31 20:35:12,2016-12-31 20:37:20,2016-12-31 20:48:26,2016-12-31 21:35:30,2016-12-31 21:35:30,0,1
2016-12-31 20:08:19,271500,6653500,H,Ambulanse,2016-12-31 20:17:18,2016-12-31 20:28:35,NaN,2016-12-31 21:03:00,2016-12-31 21:03:00,1,0
2016-12-31 20:28:57,263500,6651500,H,Ambulanse,2016-12-31 20:41:16,2016-12-31 20:42:34,2016-12-31 21:15:00,2016-12-31 21:30:53,2016-12-31 21:30:53,0,1
2016-12-31 20:29:01,262500,6650500,A,Ambulanse,2016-12-31 20:31:20,2016-12-31 20:33:54,2016-12-31 20:52:37,2016-12-31 21:17:17,2016-12-31 21:17:17,0,1
2016-12-31 20:30:37,263500,6648500,A,Ambulanse,2016-12-31 20:32:41,2016-12-31 20:39:31,2016-12-31 21:29:24,2016-12-31 21:50:00,2016-12-31 21:50:00,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:51:06,261500,6650500,A,Ambulanse,2018-12-31 19:53:48,2018-12-31 19:59:27,2018-12-31 20:33:15,2018-12-31 21:09:47,2018-12-31 21:09:47,0,1
2018-12-31 19:53:05,257500,6642500,H,Ambulanse,2018-12-31 20:48:22,2018-12-31 21:01:41,2018-12-31 21:33:17,2018-12-31 22:26:31,2018-12-31 22:26:31,0,1
2018-12-31 19:56:19,260500,6653500,A,Ambulanse,2018-12-31 19:58:34,2018-12-31 20:03:12,2018-12-31 20:22:33,2018-12-31 20:43:33,2018-12-31 20:43:33,0,1


# Build heatmap

In [7]:
counts = incidents.groupby(['xcoor', 'ycoor'], as_index=False).size()
counts['counts'] = counts['size']
counts.drop(['size'], axis=1, inplace=True)

empty_cells = pd.read_csv('data/empty_cells.csv', encoding='utf-8', index_col=0)
empty_cells = empty_cells[['X', 'Y']].rename(columns={'X': 'xcoor', 'Y': 'ycoor'})
counts = pd.concat([counts, empty_cells.assign(counts=0)]) 

counts.sort_values('counts')

,xcoor,ycoor,counts
22780006662000,278500.0,6662500.0,0
22890006700000,289500.0,6700500.0,0
22950006700000,295500.0,6700500.0,0
22970006700000,297500.0,6700500.0,0
22980006700000,298500.0,6700500.0,0
...,...,...,...
306,263500.0,6649500.0,4640
283,262500.0,6651500.0,4807
520,276500.0,6650500.0,7854
251,260500.0,6653500.0,7964


## Save as geojson file

In [8]:
features = counts.apply(lambda row: geojson_tools.centroid_to_geojson_square(*row), axis=1).tolist()
geojson_file_name = 'data/grid.geojson'

with open(geojson_file_name, 'w', encoding='utf8') as file:
        geojson.dump(geojson.FeatureCollection(features), file)

## Render map and save to image

In [9]:
if should_save:
    
    folium_map = folium.Map(
        width=400,
        location=[60.3, 8.6],
        tiles='cartodbpositron',
        zoom_start=8,
        zoom_control=False,
    )
    
    folium.GeoJson(geojson_file_name, name='geojson', style_function=map_tools.style_function).add_to(folium_map)

    # Center map on data
    south_west, north_east = list(zip(counts.xcoor.agg(['min', 'max']), counts.ycoor.agg(['min', 'max'])))
    south_west, north_east = utm_to_longitude_latitude(south_west)[::-1], utm_to_longitude_latitude(north_east)[::-1]

    folium_map.fit_bounds([south_west, north_east])

    image_data = folium_map._to_png(delay=7)
    heatmap = Image.open(io.BytesIO(image_data))
    heatmap.save('../output/visualization/heatmap.png')

    map_tools.plot_colors_as_legend(title='Aggregated incidents', file_path='../output/visualization/heatmap_legend.png')

## Number of grid cells per category

In [10]:
count_logs = counts[counts.counts > 0]['counts'].apply(np.log10)
log_groups = pd.DataFrame(count_logs.apply(math.floor).value_counts())
log_groups = pd.concat([pd.DataFrame([counts[counts.counts == 0]['counts'].shape[0]], columns=['counts']), log_groups], ignore_index=True)

color_map = map_utils.get_colors()
log_groups.index=color_map.keys()
edgecolors = ['#00000000'] *  len(color_map)
edgecolors[0] = 'grey'

ax = sns.barplot(data=log_groups, x=log_groups.counts, y=log_groups.index, palette=color_map.values(), edgecolor=edgecolors)
ax.set(xlabel='Number of grid cells', ylabel='Number of incidents')

bars = ax.patches
labels = log_groups.counts

for bar, label in zip(bars, labels):
    width = bar.get_width()
    height = bar.get_height()
    y = bar.get_y()
    ax.text(width + 150, y + height / 2, label, ha='center')

sns.despine()
plt.tight_layout()

if should_save:
    plt.savefig('../output/visualization/number_of_grid_cells.pdf', dpi=600)

NameError: name 'map_utils' is not defined

# Base station clustering

In [11]:
grids = pd.read_csv('data/grid_zones.csv', index_col=0)
grids

,pop_tot,year,easting,northing,base_station
SSBID1000M,,,,,
22390006640000,318,2017,239500,6640500,12
22390006643000,7,2017,239500,6643500,12
22400006638000,480,2017,240500,6638500,12
22400006639000,453,2017,240500,6639500,12
22400006640000,826,2017,240500,6640500,12
...,...,...,...,...,...
23210006639000,8,2017,321500,6639500,3
23210006641000,2,2017,321500,6641500,3
23210006648000,2,2017,321500,6648500,3


In [12]:
empty_cells = pd.read_csv('data/empty_cells.csv', encoding='utf-8', index_col=0)
empty_cells = empty_cells[['X', 'Y']].rename(columns={'X': 'easting', 'Y': 'northing'})
empty_cells['easting'] = empty_cells['easting'].astype(int)
empty_cells['northing'] = empty_cells['northing'].astype(int)

grids = grids[['easting', 'northing', 'base_station']]

grids = pd.concat([grids, empty_cells.assign(base_station=19)])
grids

,easting,northing,base_station
22390006640000,239500,6640500,12
22390006643000,239500,6643500,12
22400006638000,240500,6638500,12
22400006639000,240500,6639500,12
22400006640000,240500,6640500,12
...,...,...,...
23010006698000,301500,6698500,19
23000006699000,300500,6699500,19
23000006700000,300500,6700500,19
23000006701000,300500,6701500,19


In [13]:
base_stations = pd.read_csv('data/base_stations.csv', encoding='utf-8', index_col=0)
base_stations = base_stations[['easting', 'northing']]
base_stations

,easting,northing
id,,
0,287187,6692448
1,304206,6669953
2,286455,6671754
3,307577,6642937
4,275840,6650643
5,270631,6663254
6,267085,6651035
7,262948,6649765
8,261774,6652003


In [14]:
grid_zonesgeojson_grid_zones_file = 'data/.geojson'

squares = grids.apply(lambda row: geojson_tools.centroid_to_geojson_square(*row), axis=1).tolist()
points = base_stations.apply(lambda row: geojson_tools.centroid_to_geojson(*row), axis=1).tolist()

features = squares # + points

with open(geojson_grid_zones_file, 'w', encoding='utf8') as file:
        geojson.dump(geojson.FeatureCollection(features), file)
    
folium_map = folium.Map(
    width=400,
    location=[60.3, 8.6],
    tiles='cartodbpositron',
    zoom_start=8,
    zoom_control=False,
)

folium.GeoJson(geojson_grid_zones_file, name='geojson', style_function=styles.zone_styles).add_to(folium_map)

for point in points:

    folium.CircleMarker(
        location=point['geometry']['coordinates'][::-1],
        radius=3,
        color='#000000',
        fill=True,
        fill_color='#000000',
    ).add_to(folium_map)

# Center map on data
south_west, north_east = list(zip(grids.easting.agg(['min', 'max']), grids.northing.agg(['min', 'max'])))
south_west, north_east = utm_to_longitude_latitude(south_west)[::-1], utm_to_longitude_latitude(north_east)[::-1]

folium_map.fit_bounds([south_west, north_east])

image_data = folium_map._to_png(delay=7, driver=webdriver.Chrome(ChromeDriverManager().install()))
zones_image = Image.open(io.BytesIO(image_data))

# Crop image
width, height = zones_image.size
zones_image = zones_image.crop((0, 0, 400, height))

zones_image.save('../output/visualization/grid_zones.png')

[WDM] - Downloading: 100%|██████████| 7.95M/7.95M [00:01<00:00, 5.42MB/s]
/var/folders/df/lkmqtknx6n54pdp75t2xjllw0000gn/T/ipykernel_94426/482984918.py:37: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  image_data = folium_map._to_png(delay=7, driver=webdriver.Chrome(ChromeDriverManager().install()))


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.87)
Stacktrace:
0   chromedriver                        0x000000010330f0fc chromedriver + 4223228
1   chromedriver                        0x0000000103297284 chromedriver + 3732100
2   chromedriver                        0x0000000102f4b5c8 chromedriver + 275912
3   chromedriver                        0x0000000102f257b0 chromedriver + 120752
4   chromedriver                        0x0000000102fac0f8 chromedriver + 671992
5   chromedriver                        0x0000000102fbeb18 chromedriver + 748312
6   chromedriver                        0x0000000102f79748 chromedriver + 464712
7   chromedriver                        0x0000000102f7a7f0 chromedriver + 468976
8   chromedriver                        0x00000001032dfe08 chromedriver + 4029960
9   chromedriver                        0x00000001032e3698 chromedriver + 4044440
10  chromedriver   

In [ ]:
import coordinate_converter
import folium.plugins

folium_map = folium.Map(
    width=400,
    location=[60.3, 8.6],
    tiles='cartodbpositron',
    zoom_start=10,
    zoom_control=False,
)

example_allocation = [2, 1, 2, 4, 4, 3, 1, 4, 0, 2, 2, 3, 2, 2, 3, 3, 3, 1, 3]

locations = base_stations.apply(lambda row: coordinate_converter.utm_to_longitude_latitude(row)[::-1], axis=1).tolist()

for i, location in enumerate(locations):
    if (example_allocation[i] != 0):
        folium.plugins.MarkerCluster(locations=([location] * example_allocation[i])).add_to(folium_map)

# Center map on data
south_west, north_east = list(zip(grids.easting.agg(['min', 'max']), grids.northing.agg(['min', 'max'])))
south_west, north_east = utm_to_longitude_latitude(south_west)[::-1], utm_to_longitude_latitude(north_east)[::-1]

folium_map.fit_bounds([south_west, north_east])

image_data = folium_map._to_png(delay=7, driver=webdriver.Chrome(ChromeDriverManager().install()))
zones_image = Image.open(io.BytesIO(image_data))

# Crop image
width, height = zones_image.size
zones_image = zones_image.crop((0, 0, 400, height))

zones_image.save('../output/visualization/allocation.png')

/var/folders/df/lkmqtknx6n54pdp75t2xjllw0000gn/T/ipykernel_90468/2639162636.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  image_data = folium_map._to_png(delay=7, driver=webdriver.Chrome(ChromeDriverManager().install()))


KeyboardInterrupt: 